# Semigroup and Monoid

## Outline

* The mathematics

* Definition of Semigroup in Haskell

* Semigroup example

* Definition of Monoid in Haskell

* Monoid examples

In this lesson, we will add another layer of abstraction that is commonly used in Haskell. To give intuition to the reasons why this is done, we first will introduce these concepts from mathematics. After that, we will see how Haskell implements these ideas. The covered ideas are semigroups and monoids.

## The mathematics
At its core, mathematics tries to describe objects (like types) and how these objects relate to each other  (functions). Here, some classes of object have an extra certain structure to them. For example, if we consider the integers `(... -2, -1, 0, 1, 2, ...)`, we can add these and end up again with an integer. Additionally, the addition operator on these integers has an extra property, the order of adding three integers does not matter! Or more symbolic,
```
(a+b)+c = a+(b+c)
```
We call this property associativity. If objects, like the integers in the example, have such an associative operator, we call the **two** together a semigroup. Semigroups are common, and we have seen examples of these already, to name a few pairs

Objects | Operator | Associative property
--- | --- | ---
`Integers` | `(*) :: Integer -> Integer -> Integer` | `(a*b)*c = a*(b*c)`
`Integers` | `(+) :: Integer -> Integer -> Integer` | `(a+b)+c =  a+(b+c)`
`String` | `++ :: String -> String -> String ` | `(a ++ b) ++ c = a ++ (b ++ c)`
`Bool` | `(||) :: Bool -> Bool -> Bool ` | `(a || b) || c = a || (b || c)`
`Bool` | `(&&) :: Bool -> Bool -> Bool ` | `(a && b ) && c = a && (b && c)`

Note that there may be multiple associative operators available for a given object that together make them a semigroup! In Haskell, these pairs of objects and operators are captured in the `Semigroup` class.

Notice all of the above examples in the table have an extra thing in common, they have an element that under their operator leaves the other entry unaltered, or more explicit for each

Objects | Associative operator | Identity
--- | --- | ---
`Integers` | `(*) :: Integer -> Integer -> Integer` | `a*1 = a = 1*a`
`Integers` | `(+) :: Integer -> Integer -> Integer` | `a+0 = a =  a+(b+c)`
`String` | `++ :: String -> String -> String ` | `a ++ "" = a = "" ++ a`
`Bool` | `(||) :: Bool -> Bool -> Bool` | `a || False = a = False || a`
`Bool` | `(&&) :: Bool -> Bool -> Bool ` | `a && True = a = True && a`

We call this **unique** object that leave other objects unchanged the identity element. If a semigroup has such an element the pair, objects and associative operator, are called a monoid. This does not mean that all semigroups are always monoids! For example, the even numbers with associative operator `(*)` does not have the identity element as an object, so it cannot be made into a monoid.

## Definition of Semigroup in Haskell
Semigroup is a type class that provides the associative operator `<>` that is used for combining the variables of same type. Here is the type signature,
```haskell
(<>) :: Semigroup a => a -> a -> a
```

Some of the Haskell types have a built-in instance of Semigroup, lists [] for example have this. The more general `<>` operator behaves the same as the `++` operator here.

In [ ]:
[1] <> [2] -- gives you [1,2]
"a" <> "b" -- gives you "ab"

We saw that the integers do not have one unique way of implementing a semigroup, both `*` and `+` could be used for the `<>` operator. This is why Haskell provides no built-in semigroup instance for the integers. But we can implement a semigroup ourselves as,

In [ ]:
instance Semigroup Integer where
   (<>) x y = x + y

29 <> 31

## Semigroup example

Let's define our own type called Game where we define 4 options: Rock, Paper, Scissors and Mistake. When mixing them we play by standard rules (https://en.wikipedia.org/wiki/Rock_paper_scissors) only we add the Mistake element that is the case where someone did not make a clear choice. And mixing Mistake with any of the other 3 choices gives the other choice. 

In [ ]:
data Game = Rock | Paper | Scissors | Mistake deriving (Show, Eq)

instance Semigroup Game where
    (<>) Rock Paper = Paper
    (<>) Paper Rock = Paper
    (<>) Rock Scissors = Rock
    (<>) Scissors Rock = Rock
    (<>) Paper Scissors = Scissors
    (<>) Scissors Paper = Scissors
    (<>) a b | a == b = a
             | a == Mistake = b
             | b == Mistake = a

c1, c2, c3 :: Game
c1 = Rock
c2 = Paper
c3 = Mistake

print $ c1 <> c2
print $ c2 <> c2
print $ c1 <> c3
print $ c3 <> c3

## Definition of Monoid

As we saw in the mathematics section, a Monoid is an extension of the Semigroup type class, so it is also used for combining variables of the same type. The Monoids definition,
```haskell
class Semigroup a => Monoid a where
  mempty :: a
  mappend :: a -> a -> a
  mconcat :: [a] -> a
```

The `mappend` function has the same type as the `<>` operator from Semigroup, so if you define an instance for Semigroup, you can simply say `mappend = <>`. The `mempty` operator is used as the identity element, which means that combined with any other element, it does not change the other element. `mconcat` is used for combining multiple elements together with `mapend`. The minimal complete definition for the Monoid type class are only the first two operators, as we saw in the math section.

Lists are also an instance of Monoid, so you can use the Monoid operators as follows,

In [ ]:
[1] `mappend` [2] -- gives you [1,2]
"a" `mappend` "b" -- gives you "ab"
[1] `mappend` mempty -- gives you [1]
mconcat ["a","b","c"] -- gives you "abc"

Here is how you would implement Monoid for Integer if a Semigroup instance already exists

In [ ]:
instance Monoid Integer where
    mempty = 0
    mappend = (<>)
    
1 `mappend` 10

There are the laws that govern the Monoid type class:

* mappend mempty x = x.

* mappend x mempty = x.

* mappend x (mappend y z) = mappend (mappend x y) z.

* mconcat = foldr mappend mempty

You have to take care of the first three when you implement a Monoid instance. The fourth is just a definition of mconcat that you get for free.

## Monoid examples

Let's pick up on the Semigroup example and try to implement a Monoid class for it.

In [ ]:
instance Monoid Game where
    mempty = Mistake
    mappend = (<>)

print $ mconcat [Paper, Mistake, Rock, Scissors, Mistake] 

Notice that the previous code returns Paper because `mconcat [Paper, Mistake, Rock, Scissors, Mistake]` is equal to `Paper <> (Mistake <> (Rock <> (Scissors <> Mistake)))`. You can also directly define the mappend operations instead of defining the <> operations in the Semigroup class.

Let's look now at another example. You have a six sided cube where you have the number 1 one time, the number 2 two times and number 3 three times on it. Now let's create a Cube type with these three options and an instance of Semigroup and Monoid that tells you the probability of the event when you combine two possible number outcomes. 

In [ ]:
data CubeProbability = CProb String Rational

instance Show CubeProbability where
    show (CProb a b) = show a ++ "|" ++ show (fromIntegral (round (fromRational b * 100000)) / 1000) ++ "%"

instance Semigroup CubeProbability where
    (<>) cprob1 (CProb "" 0) = cprob1
    (<>) (CProb "" 0) cprob2 = cprob2 
    (<>) (CProb num1 prob1) (CProb num2 prob2) = CProb (num1 <> "-" <> num2) (prob1*prob2)

instance Monoid CubeProbability where
    mempty = CProb "" 0
    mappend = (<>)

data Cube = One | Two | Three

cube2CProb :: Cube -> CubeProbability
cube2CProb cube = case cube of 
    One -> CProb "1" (1/6)
    Two -> CProb "2" (1/3)
    Three -> CProb "3" (1/2)

case1 :: [Cube]
case1 = [Two, One, One, Three, Two]

showProbability :: [Cube] -> CubeProbability
showProbability scenario = mconcat $ map cube2CProb scenario

-- Try defining a case two and see its probability
print $ showProbability case1

## Recap

In this lesson we've discussed:

- the definition and usage of Semigroup type class
- the definition and usage of Monoid type class